In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
text_data = [
    "The quick brown fox jumps over the lazy dog",
    "One fish two fish red fish blue fish",
    "To be or not to be that is the question"
]


In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)
total_words = len(tokenizer.word_index) + 1


In [ ]:

# Create input sequences using tokenized words
input_sequences = []
for line in text_data:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)


In [ ]:
# Pad sequences for equal length
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))


In [ ]:
predictors, label = input_sequences[:,:-1], input_sequences[:,-1]
label = tf.keras.utils.to_categorical(label, num_classes=total_words)


In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(GRU(150))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
model.fit(predictors, label, epochs=100, verbose=1)


Epoch 1/100
1/1 [==============================] - 3s 3s/step - loss: 3.0432 - accuracy: 0.0833
Epoch 2/100
1/1 [==============================] - 0s 22ms/step - loss: 3.0284 - accuracy: 0.2917
Epoch 3/100
1/1 [==============================] - 0s 23ms/step - loss: 3.0134 - accuracy: 0.2500
Epoch 4/100
1/1 [==============================] - 0s 25ms/step - loss: 2.9979 - accuracy: 0.2500
Epoch 5/100
1/1 [==============================] - 0s 21ms/step - loss: 2.9816 - accuracy: 0.2083
Epoch 6/100
1/1 [==============================] - 0s 22ms/step - loss: 2.9643 - accuracy: 0.2083
Epoch 7/100
1/1 [==============================] - 0s 21ms/step - loss: 2.9457 - accuracy: 0.1667
Epoch 8/100
1/1 [==============================] - 0s 22ms/step - loss: 2.9252 - accuracy: 0.1667
Epoch 9/100
1/1 [==============================] - 0s 27ms/step - loss: 2.9026 - accuracy: 0.1667
Epoch 10/100
1/1 [==============================] - 0s 24ms/step - loss: 2.8773 - accuracy: 0.1667
Epoch 11/100
1/1 [===

In [ ]:
# Function to predict the next word
def predict_next_word(seed_text, next_words):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text


In [ ]:
def predict_next_word(seed_text, next_words):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)[0]
        predicted_index = np.argmax(predicted_probs)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text


In [ ]:
print(predict_next_word("to be", 1))

to be or
